In [10]:
# Install required packages (run once)
%pip install requests pyjwt python-dotenv --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import requests
import jwt
import uuid
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get all variables
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
current_datasource_luid = os.getenv('DATASOURCE_LUID')

print("=== Environment Variables Check ===")
print(f"TABLEAU_DOMAIN: '{tableau_server}'")
print(f"TABLEAU_SITE: '{tableau_site}'")
print(f"TABLEAU_API_VERSION: '{tableau_api_version}'")
print(f"TABLEAU_USER: '{tableau_user}'")
print(f"TABLEAU_JWT_CLIENT_ID: '{tableau_jwt_client_id}'")
print(f"TABLEAU_JWT_SECRET_ID: '{tableau_jwt_secret_id}'")
print(f"TABLEAU_JWT_SECRET: {'✅ Found' if tableau_jwt_secret else '❌ Missing'}")
print(f"DATASOURCE_LUID: '{current_datasource_luid}'")

# Fix domain format if needed
if tableau_server and tableau_server.startswith('https://'):
    tableau_server = tableau_server.replace('https://', '')
    print(f"🔧 Fixed TABLEAU_DOMAIN: {tableau_server}")

# Check for missing variables
missing_vars = []
if not tableau_server: missing_vars.append('TABLEAU_DOMAIN')
if not tableau_site: missing_vars.append('TABLEAU_SITE')
if not tableau_jwt_client_id: missing_vars.append('TABLEAU_JWT_CLIENT_ID')
if not tableau_jwt_secret_id: missing_vars.append('TABLEAU_JWT_SECRET_ID')
if not tableau_jwt_secret: missing_vars.append('TABLEAU_JWT_SECRET')
if not tableau_api_version: missing_vars.append('TABLEAU_API_VERSION')
if not tableau_user: missing_vars.append('TABLEAU_USER')

if missing_vars:
    print(f"\n❌ Missing environment variables: {', '.join(missing_vars)}")
    print("Please check your .env file!")
else:
    print("\n✅ All required environment variables found!")

=== Environment Variables Check ===
TABLEAU_DOMAIN: 'https://10ax.online.tableau.com'
TABLEAU_SITE: 'beforeinterview'
TABLEAU_API_VERSION: '3.21'
TABLEAU_USER: 'napaankur@gmail.com'
TABLEAU_JWT_CLIENT_ID: 'e355fd20-e78e-4fae-8c1a-442fa76d5a2d'
TABLEAU_JWT_SECRET_ID: 'cf23e1ae-0e7b-4ecd-8981-2bf68259cf9c'
TABLEAU_JWT_SECRET: ✅ Found
DATASOURCE_LUID: '8507283'
🔧 Fixed TABLEAU_DOMAIN: 10ax.online.tableau.com

✅ All required environment variables found!


In [3]:
def test_server_connection():
    """Test basic connection to Tableau server"""
    try:
        test_url = f"https://{tableau_server}/api/{tableau_api_version}/serverinfo"
        print(f"Testing connection to: {test_url}")
        
        response = requests.get(test_url, timeout=10)
        
        if response.status_code == 200:
            print("✅ Server is reachable!")
            return True
        else:
            print(f"❌ Server returned status {response.status_code}")
            return False
            
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to server - check your TABLEAU_DOMAIN")
        return False
    except Exception as e:
        print(f"❌ Connection error: {e}")
        return False

# Test server connection
server_ok = test_server_connection()

Testing connection to: https://10ax.online.tableau.com/api/3.21/serverinfo
✅ Server is reachable!


In [4]:
def create_jwt_token():
    """Create JWT token for authentication"""
    try:
        payload = {
            "iss": tableau_jwt_client_id,
            "exp": datetime.utcnow() + timedelta(minutes=5),
            "jti": str(uuid.uuid4()),
            "aud": "tableau", 
            "sub": tableau_user,
            "scp": ["tableau:datasources:read"]
        }
        
        token = jwt.encode(payload, tableau_jwt_secret, algorithm="HS256")
        print("✅ JWT token created successfully!")
        return token
        
    except Exception as e:
        print(f"❌ JWT token creation failed: {e}")
        return None

# Create JWT token
jwt_token = create_jwt_token()

✅ JWT token created successfully!


C:\Users\napaa\AppData\Local\Temp\ipykernel_12216\2469955053.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "exp": datetime.utcnow() + timedelta(minutes=5),


In [5]:
def test_authentication():
    """Test authentication with Tableau server"""
    if not jwt_token:
        print("❌ No JWT token available")
        return None
        
    try:
        auth_url = f"https://{tableau_server}/api/{tableau_api_version}/auth/signin"
        
        signin_payload = {
            "credentials": {
                "jwt": jwt_token,
                "site": {"contentUrl": tableau_site}
            }
        }
        
        print(f"Attempting authentication to: {auth_url}")
        print(f"Site: '{tableau_site}'")
        print(f"User: '{tableau_user}'")
        
        response = requests.post(auth_url, json=signin_payload)
        
        print(f"Response Status: {response.status_code}")
        
        if response.status_code == 200:
            auth_token = response.json()['credentials']['token']
            print("✅ Authentication successful!")
            return auth_token
        elif response.status_code == 401:
            print("❌ Authentication failed (401)")
            if "10083" in response.text:
                print("   Error 10083: Check if Connected App is ENABLED")
            elif "10001" in response.text:
                print("   Error 10001: Connected App configuration issue")
            print(f"   Response: {response.text}")
            return None
        else:
            print(f"❌ Unexpected response: {response.status_code}")
            print(f"   Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Authentication error: {e}")
        return None

# Test authentication
auth_token = test_authentication()

Attempting authentication to: https://10ax.online.tableau.com/api/3.21/auth/signin
Site: 'beforeinterview'
User: 'napaankur@gmail.com'
Response Status: 401
❌ Authentication failed (401)
   Error 10083: Check if Connected App is ENABLED
   Response: <?xml version='1.0' encoding='UTF-8'?><tsResponse xmlns="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api https://help.tableau.com/samples/en-us/rest_api/ts-api_3_21.xsd"><error code="401001"><summary>Signin Error</summary><detail>Error signing in to Tableau Server (10083)</detail></error></tsResponse>


In [6]:
def list_all_datasources():
    """List all available data sources"""
    if not auth_token:
        print("❌ No authentication token available")
        return None
    
    try:
        datasources_url = f"https://{tableau_server}/api/{tableau_api_version}/sites/{tableau_site}/datasources"
        headers = {"X-Tableau-Auth": auth_token}
        
        print(f"Fetching data sources from: {datasources_url}")
        response = requests.get(datasources_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            datasources = data['datasources']['datasource']
            
            print(f"✅ Found {len(datasources)} data sources:")
            print("=" * 80)
            
            for i, ds in enumerate(datasources, 1):
                print(f"{i:2d}. Name: {ds['name']}")
                print(f"     LUID: {ds['id']}")
                print(f"     Type: {ds.get('type', 'Unknown')}")
                print(f"     Created: {ds.get('createdAt', 'Unknown')[:10]}")
                print("-" * 60)
            
            return datasources
        else:
            print(f"❌ Failed to get data sources: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error listing data sources: {e}")
        return None

# List all data sources
all_datasources = list_all_datasources()

❌ No authentication token available


In [9]:
def debug_jwt_creation():
    """Debug JWT token creation"""
    try:
        print("=== JWT Token Debug ===")
        
        # Check JWT library
        print(f"JWT library version: {jwt.__version__}")
        
        # Create payload
        payload = {
            "iss": tableau_jwt_client_id,
            "exp": datetime.utcnow() + timedelta(minutes=5),
            "jti": str(uuid.uuid4()),
            "aud": "tableau",
            "sub": tableau_user,
            "scp": ["tableau:datasources:read"]
        }
        
        print("JWT Payload:")
        for key, value in payload.items():
            if key == "exp":
                print(f"  {key}: {value} ({value.isoformat()})")
            else:
                print(f"  {key}: {value}")
        
        # Create token
        token = jwt.encode(payload, tableau_jwt_secret, algorithm="HS256")
        print(f"JWT Token created successfully (length: {len(token)})")
        print(f"Token preview: {token[:50]}...")
        
        return token
        
    except Exception as e:
        print(f"❌ JWT creation error: {e}")
        return None

# Test JWT creation
test_token = debug_jwt_creation()

=== JWT Token Debug ===
JWT library version: 2.10.1
JWT Payload:
  iss: e355fd20-e78e-4fae-8c1a-442fa76d5a2d
  exp: 2025-05-24 15:22:14.650595 (2025-05-24T15:22:14.650595)
  jti: 28836614-83b1-4ea8-8741-aaefefedef52
  aud: tableau
  sub: napaankur@gmail.com
  scp: ['tableau:datasources:read']
JWT Token created successfully (length: 331)
Token preview: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJlM...


C:\Users\napaa\AppData\Local\Temp\ipykernel_5368\1669450705.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "exp": datetime.utcnow() + timedelta(minutes=5),


In [5]:
# Test authentication
auth_token = get_auth_token()

if auth_token:
    print(f"Auth token received: {auth_token[:20]}...")
else:
    print("Authentication failed. Check your credentials.")

C:\Users\napaa\AppData\Local\Temp\ipykernel_5368\1364815202.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "exp": datetime.utcnow() + timedelta(minutes=5),  # Back to the working version


Attempting to connect to: https://10ax.online.tableau.com/api/3.21/auth/signin
❌ Auth failed: 401
Response: <?xml version='1.0' encoding='UTF-8'?><tsResponse xmlns="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api https://help.tableau.com/samples/en-us/rest_api/ts-api_3_21.xsd"><error code="401001"><summary>Signin Error</summary><detail>Error signing in to Tableau Server (10083)</detail></error></tsResponse>
Authentication failed. Check your credentials.
